In [1]:
import matplotlib.pyplot as plt
from itertools import cycle
from tabulate import tabulate
from pychegp.atmosphere.atmos import load_species_masses
from pychegp.constants import eps_dp
#import pychegp.equil.md_ace as md_ace
from pychegp.parameter.parameters import create_atmosphere, create_star, create_planet, create_cross_sections, \
    create_chemical_network, create_chemistry, create_ode
import os
from pychegp.parameter.parameters import transform
from pychegp.pychegp import ChemicalKinetics
import numpy as np
import logging
import glob
from pychegp import root_logger
from scipy.constants import G
import numpy as np
import pandas as pd
root_logger.handlers = []
root_logger.addHandler(logging.StreamHandler())
root_logger.setLevel(logging.INFO)

In [2]:
def calculate_gravity(M,r):
    from scipy.constants import G
    M = M * 1.899e+27
    r = r * 7.149e+7
    g = (G * M) / (np.square(r))
    return g

In [3]:
def load_configuration_taurex(name, path_conf = 'data'):
    '''Load configuration for specific system from ACE taurex
    name - name of the system
    masses - general list of masses for species
    fm - input array for kinetic chemistry
    species - list of species, for taurex we added 6 species (3 condensations) with value 1e-30
    chemistry dataframe contais pressure as well so needs to be removed before creating fm/species list
    planet temperature - isothermal
    pressure - output from taurex is in pascal, here we need mbar
    altitude - output from taurex is in meters, here we need km
    '''
    
    from pychegp.constants import k_boltz_si
    
    #load all csv
    # interested in specific system
    conf = pd.read_csv(f'{path_conf}/configuration/system_configuration.csv')
    conf = conf[conf['System_ID'] == name].iloc[0].to_dict()
    layers = conf['Layers']
    chemistry = pd.read_csv(f'{path_conf}/mix_profiles/{name}.csv')
    
    # load masses -> assumption all masses (per molecule) are same, could load from pychegp?
    masses = load_array_3d('masses.npy')
    
    # create list of species -> needs to remove pressure and altitude
    species = chemistry.columns.to_list()
    species.remove('Pressure')
    species.remove('Altitude_m')
    
    # create fm array -> new version takes npy array
    fm = load_array_3d(f'{path_conf}/mix_profiles/{name}.npy')
    
    # BOA -> TOA
    temperature = np.full((layers,), float(conf['isothermal_T']))  # K
    pressure = np.array(chemistry['Pressure'].to_list())/100  # mbar
    altitude = np.sort(np.array(chemistry['Altitude_m'].to_list())/1000) # km
    density = pressure*1e-3/(k_boltz_si*temperature)
    Kzz = conf.get('kzz', 1e9)
    Kzz = np.full_like(altitude, Kzz)
    
    
    # with taurex we use planet mass but freckll needs gravity
    go = calculate_gravity(conf['planet_mass'],
                           conf['planet_radius'])
    
    # putting everything into dictionary
    configuration = {
        'Atmosphere': {
            'pressure': pressure, 
            'temperature': temperature,
            'altitude': altitude,
            'density': density,
            'kzz': Kzz,
        },
        'Planet': {
            'planet_name': conf['System_ID'],
            'radius_jup':conf['planet_radius'],
            'mass_jup':conf['planet_mass'],
            'temp_K':conf['isothermal_T'],
            'radius_m': conf['planet_radius'] * 7.149e7, # Radius in m
            'semi_major_axis_au': conf['semi_major_axis_au'], # Semi major axis in AU
            'go': go,  # Gravity in m/s**2
        },   
            'Chemical Network': {
            'reduced': False,
        },
        'ODE': {
            'solver': 'vode',
            'atol': 1e-19, # Absolute tolerance
            'rtol': 1e-3, # Relative tolerance
            'dn_stop': 0.01, # Stop when relative change in abundances (dn) is small enough.
            'dndt_stop': 1e-8, # Stop when relative change in abundance with respect to time (dn/dt) is small enough.
            'end_time': 1e10, # at what t to stop the simulation defaults to 1e20
        }
    }
    return configuration, fm, species, masses

In [6]:
def save_array_3d(filename, arr):
    with open(filename, 'wb') as f:
        np.save(f, arr)

def load_array_3d(filename):
    with open(filename, 'rb') as f:
        arr = np.load(f)
    return arr

def save_file(results, name, save_path = 'data'):
    
    files = [f'{save_path}/ode_results/', f'{save_path}/ode_results_time_steps/']
    for directory in files:
        if not os.path.exists(directory):
            os.makedirs(directory)
   
    chem_df = pd.DataFrame(results['oderesult'][0][-1].T, columns=results['species'])
    chem_df['Pressure'] = results['P']
    chem_df['Temp'] = results['T']
    
    chem_df.to_csv(f'{save_path}/ode_results/{name}.csv', index=False)
    
    save_array_3d(f'{save_path}/ode_results_time_steps/{name}.npy', np.array(results['oderesult'][0]))

In [41]:
def freckll_chemistry(name, save=True, path_conf='data', save_path='data', freckll=False):
     
    if freckll:
        configuration = load_configuration(name, path=path_conf)
        masses = load_masses(name, path=path_conf)
        fm, species = load_chemistry(name, path=path_conf)
    if not freckll:
        configuration,fm, species, masses = load_configuration_taurex(name, path_conf = path_conf)
    
    #masses = load_masses(name)
    equil_fm = np.copy(fm)
    name =  configuration['Planet']['planet_name']
    
    # atmosphere
    atmosphere_config = configuration['Atmosphere']
    altitude = atmosphere_config['altitude']
    P = atmosphere_config['pressure']
    T = atmosphere_config['temperature']
    density = atmosphere_config['density']
    Kzz = atmosphere_config['kzz']

        
    # planet
    planet_params = create_planet(configuration)
    name, radius, sma, go = planet_params
    Rm = radius/1000

    # Star
    star_params = create_star(configuration)
    in_flux = None
    if star_params[0] is None:
        root_logger.info('No stellar flux include, photodiso is disabled')
    else:
        from pychegp.photodissociation import incident_stellar_flux
        root_logger.info('Stellar flux loaded')
        in_flux = incident_stellar_flux(star_params[1], sma)

    cross_sections = create_cross_sections(configuration)
    
    cn, reduced = create_chemical_network(
        configuration, cross_sections=cross_sections)

    ode, atol, rtol, dn_stop, dndt_stop, end_time, nsteps, time_limit = create_ode(
        configuration)


    cn.set_stellar_data(star_params[0], in_flux)

    ck = ChemicalKinetics(T, P, altitude, density, Rm, go,
                          Kzz, species, fm, cn, ode, masses)
    ck.recompute_cn(ck.fm)


    output = ck.solve(np.log10(end_time), nsteps, atol=atol, rtol=rtol,
                      df_stop=dn_stop, dfdt_stop=dndt_stop, maximum_runtime=time_limit)
   
    fms, ts, conv = output

    results = {'species': ck.species, 'oderesult': output,
               'T': T, 'P': P, 'equil_fm': equil_fm}
    if save:
        save_file(results, name, save_path=save_path)
        
    #return results


In [31]:
name_list = glob.glob('data/mix_profiles/*csv')

In [33]:
for name_ace in name_list:
    name = name_ace.split('/')[-1].split('.')[0]
    print(name)
    freckll_chemistry(name)

No stellar flux include, photodiso is disabled

Loading cross-sections:
Using default cross-sections
Cross-sections complete.

Mapping /Users/sponka/anaconda3/envs/taurex2/lib/python3.7/site-packages/pychegp/data/Venot2020_Taurex/combinaison_k0_kinf_rev.dat -> k0kinf_reaction
Mapping /Users/sponka/anaconda3/envs/taurex2/lib/python3.7/site-packages/pychegp/data/Venot2020_Taurex/combinaison_k0_kinf_rev_SRI.dat -> k0kinf_SRI_reaction
Mapping /Users/sponka/anaconda3/envs/taurex2/lib/python3.7/site-packages/pychegp/data/Venot2020_Taurex/combinaison_k0_rev.dat -> k0_reaction
Mapping /Users/sponka/anaconda3/envs/taurex2/lib/python3.7/site-packages/pychegp/data/Venot2020_Taurex/decompo_k0_kinf_rev.dat -> decompo_k0kinf_reaction
Mapping /Users/sponka/anaconda3/envs/taurex2/lib/python3.7/site-packages/pychegp/data/Venot2020_Taurex/decompo_k0_kinf_rev_SRI.dat -> decompo_k0kinf_SRI_reaction
Mapping /Users/sponka/anaconda3/envs/taurex2/lib/python3.7/site-packages/pychegp/data/Venot2020_Taurex/decom

ACE_20230518150835278396_cej
(111, 100)


Photodissociation is disabled
Running 0.0 - 1.0
Photodissociation is disabled
Starting VODE integrator
/Users/sponka/anaconda3/envs/taurex2/lib/python3.7/site-packages/pychegp/kinetics.py:386: RuntimeWarning: divide by zero encountered in log10
  root_logger.info(f'ydot sum/min/max: {final.sum():4.2E} / {final.min():4.2E} / {final.max():4.2E} Time/log(Time): {t:4.2E} / {np.log10(t):4.2E}')
ydot sum/min/max: -9.10E+00 / -1.51E+00 / 1.49E+00 Time/log(Time): 0.00E+00 / -INF
ydot sum/min/max: -9.10E+00 / -1.51E+00 / 1.49E+00 Time/log(Time): 1.17E-14 / -1.39E+01
ydot sum/min/max: -9.10E+00 / -1.51E+00 / 1.49E+00 Time/log(Time): 3.71E-14 / -1.34E+01
ydot sum/min/max: -9.10E+00 / -1.51E+00 / 1.49E+00 Time/log(Time): 3.30E-14 / -1.35E+01
ydot sum/min/max: -9.09E+00 / -1.51E+00 / 1.49E+00 Time/log(Time): 5.64E-13 / -1.22E+01
ydot sum/min/max: -9.09E+00 / -1.51E+00 / 1.49E+00 Time/log(Time): 5.64E-13 / -1.22E+01
ydot sum/min/max: -9.08E+00 / -1.50E+00 / 1.48E+00 Time/log(Time): 2.42E-12 / -1.16E

ydot sum/min/max: -1.52E-01 / -6.38E-03 / 5.01E-03 Time/log(Time): 3.26E-06 / -5.49E+00
ydot sum/min/max: -1.50E-01 / -6.22E-03 / 4.96E-03 Time/log(Time): 3.87E-06 / -5.41E+00
ydot sum/min/max: -1.48E-01 / -6.08E-03 / 4.92E-03 Time/log(Time): 4.47E-06 / -5.35E+00
ydot sum/min/max: -1.45E-01 / -6.03E-03 / 4.84E-03 Time/log(Time): 5.43E-06 / -5.26E+00
ydot sum/min/max: -1.43E-01 / -5.99E-03 / 4.77E-03 Time/log(Time): 6.40E-06 / -5.19E+00
ydot sum/min/max: -1.42E-01 / -5.94E-03 / 4.70E-03 Time/log(Time): 7.36E-06 / -5.13E+00
ydot sum/min/max: -1.39E-01 / -5.87E-03 / 4.59E-03 Time/log(Time): 8.91E-06 / -5.05E+00
ydot sum/min/max: -1.37E-01 / -5.80E-03 / 4.49E-03 Time/log(Time): 1.05E-05 / -4.98E+00
ydot sum/min/max: -1.37E-01 / -5.80E-03 / 4.49E-03 Time/log(Time): 1.05E-05 / -4.98E+00
ydot sum/min/max: -1.35E-01 / -5.73E-03 / 4.40E-03 Time/log(Time): 1.20E-05 / -4.92E+00
ydot sum/min/max: -1.33E-01 / -5.66E-03 / 4.30E-03 Time/log(Time): 1.36E-05 / -4.87E+00
ydot sum/min/max: -1.30E-01 / -5

ydot sum/min/max: -2.39E-02 / -9.72E-04 / 9.53E-04 Time/log(Time): 1.14E-02 / -1.94E+00
ydot sum/min/max: -2.38E-02 / -9.65E-04 / 9.47E-04 Time/log(Time): 1.18E-02 / -1.93E+00
ydot sum/min/max: -2.36E-02 / -9.59E-04 / 9.41E-04 Time/log(Time): 1.21E-02 / -1.92E+00
ydot sum/min/max: -2.36E-02 / -9.59E-04 / 9.41E-04 Time/log(Time): 1.21E-02 / -1.92E+00
ydot sum/min/max: -2.34E-02 / -9.53E-04 / 9.35E-04 Time/log(Time): 1.25E-02 / -1.90E+00
ydot sum/min/max: -2.32E-02 / -9.47E-04 / 9.30E-04 Time/log(Time): 1.29E-02 / -1.89E+00
ydot sum/min/max: -2.31E-02 / -9.42E-04 / 9.24E-04 Time/log(Time): 1.32E-02 / -1.88E+00
ydot sum/min/max: -2.31E-02 / -9.42E-04 / 9.24E-04 Time/log(Time): 1.32E-02 / -1.88E+00
ydot sum/min/max: -2.29E-02 / -9.36E-04 / 9.18E-04 Time/log(Time): 1.36E-02 / -1.87E+00
ydot sum/min/max: -2.29E-02 / -9.36E-04 / 9.18E-04 Time/log(Time): 1.36E-02 / -1.87E+00
ydot sum/min/max: -2.27E-02 / -9.30E-04 / 9.13E-04 Time/log(Time): 1.39E-02 / -1.86E+00
ydot sum/min/max: -2.27E-02 / -9

...

ydot sum/min/max: -3.72E-04 / -4.56E-05 / 2.30E-05 Time/log(Time): 5.02E+01 / 1.70E+00
ydot sum/min/max: -3.58E-04 / -4.40E-05 / 2.22E-05 Time/log(Time): 5.27E+01 / 1.72E+00
ydot sum/min/max: -3.44E-04 / -4.27E-05 / 2.16E-05 Time/log(Time): 5.52E+01 / 1.74E+00
ydot sum/min/max: -3.32E-04 / -4.14E-05 / 2.09E-05 Time/log(Time): 5.77E+01 / 1.76E+00
ydot sum/min/max: -3.21E-04 / -4.02E-05 / 2.04E-05 Time/log(Time): 6.02E+01 / 1.78E+00
ydot sum/min/max: -3.11E-04 / -3.92E-05 / 1.98E-05 Time/log(Time): 6.26E+01 / 1.80E+00
ydot sum/min/max: -3.01E-04 / -3.82E-05 / 1.94E-05 Time/log(Time): 6.51E+01 / 1.81E+00
ydot sum/min/max: -2.92E-04 / -3.73E-05 / 1.89E-05 Time/log(Time): 6.76E+01 / 1.83E+00
ydot sum/min/max: -2.84E-04 / -3.63E-05 / 1.86E-05 Time/log(Time): 7.01E+01 / 1.85E+00
ydot sum/min/max: -2.76E-04 / -3.54E-05 / 1.84E-05 Time/log(Time): 7.25E+01 / 1.86E+00
ydot sum/min/max: -2.68E-04 / -3.47E-05 / 1.82E-05 Time/log(Time): 7.50E+01 / 1.88E+00
ydot sum/min/max: -2.57E-04 / -3.35E-05 / 1

ydot sum/min/max: -1.36E-05 / -1.23E-05 / 8.40E-06 Time/log(Time): 6.84E+02 / 2.84E+00
ydot sum/min/max: -1.16E-05 / -1.21E-05 / 8.25E-06 Time/log(Time): 7.11E+02 / 2.85E+00
ydot sum/min/max: -9.72E-06 / -1.19E-05 / 8.11E-06 Time/log(Time): 7.38E+02 / 2.87E+00
ydot sum/min/max: -7.93E-06 / -1.17E-05 / 7.97E-06 Time/log(Time): 7.65E+02 / 2.88E+00
ydot sum/min/max: -6.25E-06 / -1.15E-05 / 7.83E-06 Time/log(Time): 7.92E+02 / 2.90E+00
ydot sum/min/max: -4.65E-06 / -1.13E-05 / 7.70E-06 Time/log(Time): 8.19E+02 / 2.91E+00
ydot sum/min/max: -3.15E-06 / -1.11E-05 / 7.57E-06 Time/log(Time): 8.46E+02 / 2.93E+00
ydot sum/min/max: -3.54E-06 / -1.11E-05 / 7.57E-06 Time/log(Time): 8.46E+02 / 2.93E+00
ydot sum/min/max: -6.31E-07 / -1.09E-05 / 7.44E-06 Time/log(Time): 8.73E+02 / 2.94E+00
ydot sum/min/max: -1.38E-06 / -1.07E-05 / 7.32E-06 Time/log(Time): 9.00E+02 / 2.95E+00
ydot sum/min/max: 1.26E-06 / -1.06E-05 / 7.19E-06 Time/log(Time): 9.27E+02 / 2.97E+00
ydot sum/min/max: 2.17E-06 / -1.04E-05 / 7.0

ydot sum/min/max: 7.02E-05 / -3.70E-05 / 3.99E-05 Time/log(Time): 4.69E+03 / 3.67E+00
ydot sum/min/max: 2.85E-05 / -8.54E-06 / 7.99E-06 Time/log(Time): 4.69E+03 / 3.67E+00
ydot sum/min/max: 3.49E-05 / -6.97E-07 / 1.05E-06 Time/log(Time): 4.69E+03 / 3.67E+00
ydot sum/min/max: 3.58E-05 / -5.43E-06 / 5.60E-06 Time/log(Time): 4.97E+03 / 3.70E+00
ydot sum/min/max: 2.34E-05 / -1.69E-05 / 1.58E-05 Time/log(Time): 5.25E+03 / 3.72E+00
ydot sum/min/max: 2.90E-05 / -6.83E-06 / 6.45E-06 Time/log(Time): 5.25E+03 / 3.72E+00
ydot sum/min/max: 5.93E-05 / -2.53E-05 / 2.70E-05 Time/log(Time): 5.52E+03 / 3.74E+00
ydot sum/min/max: 1.98E-05 / -2.14E-05 / 2.02E-05 Time/log(Time): 5.80E+03 / 3.76E+00
ydot sum/min/max: 3.90E-05 / -6.09E-06 / 6.50E-06 Time/log(Time): 6.08E+03 / 3.78E+00
ydot sum/min/max: 3.02E-05 / -5.06E-06 / 4.82E-06 Time/log(Time): 6.08E+03 / 3.78E+00
ydot sum/min/max: 3.45E-05 / -2.68E-07 / 7.07E-07 Time/log(Time): 6.08E+03 / 3.78E+00
ydot sum/min/max: 3.24E-05 / -2.19E-06 / 2.10E-06 Time

ydot sum/min/max: 3.16E-05 / -7.82E-08 / 4.80E-07 Time/log(Time): 1.00E+04 / 4.00E+00
ydot sum/min/max: 3.14E-05 / -1.44E-07 / 4.78E-07 Time/log(Time): 1.00E+04 / 4.00E+00
ydot sum/min/max: 3.14E-05 / -9.54E-08 / 4.77E-07 Time/log(Time): 1.00E+04 / 4.00E+00
ydot sum/min/max: 3.13E-05 / -7.82E-08 / 4.75E-07 Time/log(Time): 1.00E+04 / 4.00E+00
ydot sum/min/max: 3.13E-05 / -7.85E-08 / 4.72E-07 Time/log(Time): 1.00E+04 / 4.00E+00
ydot sum/min/max: 3.12E-05 / -1.31E-07 / 4.61E-07 Time/log(Time): 1.01E+04 / 4.00E+00
ydot sum/min/max: 2.42E-05 / -4.92E-06 / 4.87E-06 Time/log(Time): 1.05E+04 / 4.02E+00
ydot sum/min/max: 2.18E-05 / -6.35E-06 / 6.29E-06 Time/log(Time): 1.09E+04 / 4.04E+00
ydot sum/min/max: -1.77E-05 / -3.38E-05 / 3.35E-05 Time/log(Time): 1.20E+04 / 4.08E+00
ydot sum/min/max: 4.28E-05 / -9.65E-06 / 9.81E-06 Time/log(Time): 1.20E+04 / 4.08E+00
ydot sum/min/max: 2.10E-05 / -5.72E-06 / 5.69E-06 Time/log(Time): 1.31E+04 / 4.12E+00
ydot sum/min/max: 9.41E-05 / -4.82E-05 / 4.88E-05 Tim

ydot sum/min/max: 4.27E-07 / -9.72E-10 / 9.84E-09 Time/log(Time): 1.00E+05 / 5.00E+00
ydot sum/min/max: 4.24E-07 / -9.66E-10 / 9.82E-09 Time/log(Time): 1.00E+05 / 5.00E+00
ydot sum/min/max: 4.18E-07 / -9.43E-10 / 9.76E-09 Time/log(Time): 1.00E+05 / 5.00E+00
ydot sum/min/max: 4.10E-07 / -8.96E-10 / 9.70E-09 Time/log(Time): 1.00E+05 / 5.00E+00
ydot sum/min/max: 3.97E-07 / -7.78E-10 / 9.61E-09 Time/log(Time): 1.01E+05 / 5.00E+00
ydot sum/min/max: 3.74E-07 / -6.00E-10 / 9.35E-09 Time/log(Time): 1.01E+05 / 5.01E+00
ydot sum/min/max: 3.31E-07 / -1.60E-09 / 8.51E-09 Time/log(Time): 1.03E+05 / 5.01E+00
ydot sum/min/max: 2.59E-07 / -2.09E-09 / 9.01E-09 Time/log(Time): 1.06E+05 / 5.02E+00
ydot sum/min/max: 1.66E-07 / -1.12E-08 / 1.58E-08 Time/log(Time): 1.12E+05 / 5.05E+00
ydot sum/min/max: 1.22E-07 / -1.22E-08 / 1.54E-08 Time/log(Time): 1.17E+05 / 5.07E+00
ydot sum/min/max: 5.04E-08 / -6.12E-09 / 6.16E-09 Time/log(Time): 1.27E+05 / 5.10E+00
ydot sum/min/max: -4.80E-09 / -7.83E-09 / 7.68E-09 Tim